In [57]:
import osmnx as ox, geopandas as gpd
from shapely.geometry import Point, MultiLineString

import os
BASE_DIR = os.path.dirname(os.getcwd())
os.chdir(BASE_DIR + '/src/data/')
import util
os.chdir(BASE_DIR + '/notebooks/')

In [58]:
gdf_place = ox.gdf_from_place('Cambridge, Massachusetts, USA')
polygon = gdf_place['geometry'].unary_union
cambridge = ox.core.osm_net_download(polygon)

In [59]:
elements = cambridge[0]['elements']

# Grab all the ways in a list
#ways = [x for x in elements if x['type'] == 'way']

# Turn the nodes into key value pairs where key is the node id
nodes = {x['id']: x for x in elements if x['type'] == 'node'}


In [71]:
node_counts = {}
dup_nodes = {}
# Any node that's shared by more than one way is an intersection

non_named_count = 0
way_lines = []
unnamed_lines = []
service_lines = []
for way in elements:
    if way['type'] == 'way':

        coords = []
        way_nodes = way['nodes']
    
        prev = None
        for i in range(len(way_nodes)):
            n = way_nodes[i]
            if prev:
                coords.append((
                (prev['lon'], prev['lat']),
                (nodes[n]['lon'], nodes[n]['lat']),
            ))
            prev = nodes[n]

            if n in node_counts.keys():
                dup_nodes[n] = 1

            else:
                node_counts[n] = 0
            node_counts[n] += 1
        # Make the multiline string for this way
        name = None
        if 'name' in way['tags'].keys():
            if way['tags']['highway'] in ('service', 'footway'):
                service_lines.append((MultiLineString(coords), {'name': way['tags']['name'], 'id': way['id']}))
            else:    
                way_lines.append((MultiLineString(coords), {'name': way['tags']['name'], 'id': way['id']}))
        else:
            unnamed_lines.append((MultiLineString(coords), {'id': way['id']}))
            non_named_count += 1
    

print 'Found ' + str(len(way_lines)) + ' residential roads'
print 'Found ' + str(len(unnamed_lines)) + ' unnamed roads'
print 'Found ' + str(len(service_lines)) + ' service roads or footpaths'
print "Found " + str(len(dup_nodes.keys())) + " intersections"

Found 1601 residential roads
Found 1842 unnamed roads
Found 53 service roads or footpaths
Found 4535 intersections


In [72]:
# Write all the ways out
schema = {'geometry': 'MultiLineString', 'properties': {'id': 'int', 'name': 'str'}}
util.write_shp(schema, BASE_DIR + '/osm-data/processed/maps/named_ways.shp', way_lines, 0, 1)

# Write the service roads
util.write_shp(schema, BASE_DIR + '/osm-data/processed/maps/service_ways.shp', service_lines, 0, 1)

# Write the unnamed ways out
schema = {'geometry': 'MultiLineString', 'properties': {'id': 'int'}}
print way_lines[0]
print way_lines[0][1]
util.write_shp(schema, BASE_DIR + '/osm-data/processed/maps/unnamed_ways.shp', unnamed_lines, 0, 1)


points = []

for node in dup_nodes.keys():
    points.append((
        Point(nodes[node]['lon'], nodes[node]['lat']),
        {'node_id': node}
    ))
schema = {'geometry': 'Point', 'properties': {'node_id': 'int'}}
util.write_points(points, schema, BASE_DIR + '/osm-data/processed/maps/osm_inter.shp')

(<shapely.geometry.multilinestring.MultiLineString object at 0x7f8490315dd0>, {'name': u'Columbia Street', 'id': 8604746})
{'name': u'Columbia Street', 'id': 8604746}
finished 0 of 4535
finished 500 of 4535
finished 1000 of 4535
finished 1500 of 4535
finished 2000 of 4535
finished 2500 of 4535
finished 3000 of 4535
finished 3500 of 4535
finished 4000 of 4535
finished 4500 of 4535
